In [3]:
#import necessary modules:
import os
import numpy as np
import pandas as pd
from glob import glob
from natsort import natsorted
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet
import torchvision.datasets as datasets
import cv2
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

In [5]:
#define dataset and dataloaders
train_df_src = r'\\fatherserverdw\Kevin\unstained_blank_classifier\train_df.xlsx'
train_df = pd.read_excel(train_df_src) # 1= white , 0=nonwhite, unbalanced, 79271 0's and 195376 1's. Need stratifiedgroupKfold for CV.
train_df = train_df.drop(columns="Unnamed: 0")
train_df

,imagepath,label
0,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
1,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
2,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
3,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
4,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
...,...,...
274642,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
274643,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
274644,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
274645,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1


In [6]:
# first find mean and std of dataset for image normalization:
class Unstain2StainData(Dataset):
    def __init__(self,df,transform=None):
        self.df = df
        self.directory = df["imagepath"].tolist()
        self.transform = transform

    def __len__(self):
        return len(self.directory)

    def __getitem__(self,idx):
        path = self.directory[idx]
        image = cv2.imread(path, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image = image)['image']
        return image

In [7]:
device      = torch.device('cpu')
num_workers = 0
image_size  = 384
batch_size  = 8

In [8]:
augmentations = A.Compose([A.Resize(height= image_size ,width = image_size ),
                                   A.Normalize(mean=(0,0,0), std=(1,1,1)),
                                   ToTensorV2()])

In [9]:
unstain2stain_dataset = Unstain2StainData(df = train_df, transform = augmentations)# data loader
image_loader = DataLoader(unstain2stain_dataset,
                          batch_size  = batch_size,
                          shuffle     = False,
                          num_workers = num_workers,
                          pin_memory  = True)
images = next(iter(image_loader))
print("Images have a tensor size of {}.".
      format(images.size()))

Images have a tensor size of torch.Size([8, 3, 384, 384]).


In [ ]:
# compute mean/std:
# placeholders
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs in tqdm(image_loader,colour='red'):
    psum    += inputs.sum(axis = [0, 2, 3]) # sum over axis 1
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3]) # sum over axis 1

# pixel count
count = len(train_df) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: ' + str(total_mean))
print('std:  ' + str(total_std))

 76%|███████▋  | 26239/34331 [17:53:40<5:07:13,  2.28s/it] 

In [ ]:
# add stratifiedkfold to df:
new_df_train = train_df.deepcopy()
strat_kfold = StratifiedKFold(shuffle = True, random_state = 42) #use default n_split = 5, random_state for reproducibility

#split (stratification) on is_empty and group (groupkfold) on case_number:
for each_fold, (idx1,idx2) in enumerate (strat_kfold.split(X = new_df_train, y = new_df_train['label'])):
    new_df_train.loc[idx2,'fold'] = int(each_fold) #create new fold column with the fold number (up to 5)

#check if stratification worked by grouping:
grouped = new_df_train.groupby(['fold','label']) # look how it's splitted
display(grouped.id.count())

ratio_list = []
for k in range(5):
    ratio = grouped.id.count()[k][0]/grouped.id.count()[k][1]
    ratio_list.append(ratio)
print("the ratios of the folds are: {}".format(ratio_list)) #ratios to check stratification

In [ ]:
#define transforms/image augmentation for the dataset
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(384), # efficientnetv2_s 384 x 384
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #imagenet1k weights
])

val_transform = transforms.Compose([
 # validate at 1024 x 1024, you want to use val dataset to real world application, but maybe resize to 384 if performance is bad.
    # transforms.Resize(384),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #imagenet1k weights
])

In [ ]:
# build train and valid dataset
class TrainDataSet(Dataset):
    # initialize df, label, imagepath and transforms
    def __init__(self, df, label=True, transform = None):
        self.df = df
        self.label = df["label"].tolist()
        self.imagepaths = df["imagepath"].tolist()
        self.transform = transform
    # define length, which is simply length of all imagepaths
    def __len__(self):
        return len(self.df)
    # define main function to read image and label, apply transform function and return the transformed images.
    def __getitem__(self,idx):
        image_path = self.imagepaths[idx]
        image = cv2.imread(image_path, -1)
        if self.label:
            label = self.label[idx]

In [ ]:
# #define pre-trained resNet-18 model
# model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 2)  #replace/edit output layer with a new linear layer for binary classification- blank or not blank

# use efficientnetv2 small instead, should do better than resnet18/50
model = EfficientNet.from_pretrained('efficientnetv2-s')

# Modify the last layer to output a single binary classification output
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, 1)

#define loss function, optimizer and device
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

### training loop:

In [ ]:
def save_model(epoch, model, optimizer, criterion):
    torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, 'outputs/model.pth')

In [ ]:
def save_plots(train_accuracy_list, val_accuracy_list,train_loss_list,val_loss_list):    # accuracy plots
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_accuracy_list, color='green', linestyle='-',
        label='train accuracy'
    )
    plt.plot(
        val_accuracy_list, color='blue', linestyle='-',
        label='validation accuracy'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('outputs/accuracy.png')

    # loss plots
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_loss_list, color='orange', linestyle='-',
        label='train loss'
    )
    plt.plot(
        val_loss_list, color='red', linestyle='-',
        label='validataion loss'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('outputs/loss_vs_epochs.png')

In [ ]:
#training loop
num_epochs = 10
train_loss_list, val_loss_list = [], []
train_accuracy_list, val_accuracy_list = [], []

for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_loss += loss.item()
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

    train_loss = train_loss / len(train_dataset)
    train_accuracy = train_correct / train_total
    train_loss_list.append(train_loss)
    train_accuracy_list.append(train_accuracy)

    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            val_loss += loss.item()
            val_correct +=  (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_loss = val_loss / len(val_dataset)
    val_accuracy = val_correct / val_total
    val_loss_list.append(val_loss)
    val_accuracy_list.append(val_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

save_model(epoch, model, optimizer, criterion)
save_plots(train_accuracy, val_accuracy,train_loss,val_loss)